In [ ]:
import itertools as it
import numpy as np
from numpy import random as random

# Question 1

In [ ]:
BASES = ['A', 'C', 'G', 'T']
n = 50
mu = 0.01
t = 10

We draw $n$ bases from the discrete uniform distribution on $\{A, C, G, T\}$ to get the ancestral sequence.

In [ ]:
def random_sequence(n):
    return ''.join(random.choice(BASES, n))

ancestor = random_sequence(n)
ancestor

In [ ]:
def poisson_process(lambd):
    t = 0
    lambd = 1.0 / lambd
    while True:
        t += random.exponential(lambd)
        yield t

In [ ]:
def evolve_sequence(s, mu, t):
    s = [c for c in s]
    n = len(s)
    lambd = 0.75 * mu * n
    for _ in it.takewhile(lambda x: x < t, poisson_process(lambd)):
        i = random.randint(n)
        s[i] = random.choice([b for b in BASES if b != s[i]])
    return ''.join(s)

The sequence for the first child is

In [ ]:
child1 = evolve_sequence(ancestor, mu, t)
child1

and the sequence for the second child is

In [ ]:
child2 = evolve_sequence(ancestor, mu, t)
child2

In [ ]:
def count_differences(x, y):
    return sum(1 for a, b in zip(x, y) if a == b)

The number of differences between the ancestor and child 1, the 